In [3]:
from ClasificadorGenetico import ClasificadorGenetico
from Clasificador import Clasificador
from Datos import Datos

clasificador = Clasificador()
datos = Datos("./datasets/xor.csv")

cg = ClasificadorGenetico()
cg.entrenamiento(datos.datos, datos.nominalAtributos, datos.diccionarios)

for ind in cg.population:
    print(ind)

----> 200 400
----> 100 400
----> 200 400
----> 100 400
----> 200 400
----> 100 400
----> 200 400
----> 100 400
----> 200 400
----> 100 400
----> 200 400
----> 200 400
----> 100 400
----> 0 400
----> 200 400
----> 100 400
----> 200 400
----> 200 400
----> 100 400
----> 100 400
----> 200 400
----> 100 400
----> 200 400
----> 100 400
----> 0 400
----> 100 400
----> 200 400
----> 100 400
----> 100 400
----> 100 400
----> 200 400
----> 200 400
----> 200 400
----> 200 400
----> 200 400
----> 300 400
----> 100 400
----> 100 400
----> 100 400
----> 300 400
----> 100 400
----> 100 400
----> 100 400
----> 200 400
----> 200 400
----> 200 400
----> 0 400
----> 200 400
----> 0 400
----> 200 400
[0.5, 0.25, 0.5, 0.25, 0.5, 0.25, 0.5, 0.25, 0.5, 0.25, 0.5, 0.5, 0.25, 0.0, 0.5, 0.25, 0.5, 0.5, 0.25, 0.25, 0.5, 0.25, 0.5, 0.25, 0.0, 0.25, 0.5, 0.25, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.75, 0.25, 0.25, 0.25, 0.75, 0.25, 0.25, 0.25, 0.5, 0.5, 0.5, 0.0, 0.5, 0.0, 0.5]
[[0, 1, 0], [0, 1, 1], [1, 0, 1]]
